# 11.03 - Data Wrangling con Polars

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Avanzado  
**Tiempo estimado:** 50 min

En este lab profundizaremos en operaciones de transformación de datos complejas: agregaciones, agrupaciones y uniones de datos.

In [1]:
import polars as pl
import numpy as np

# Generación de datos sintéticos de ventas
np.random.seed(42)
n = 1000
df_sales = pl.DataFrame({
    "store_id": np.random.randint(1, 10, n),
    "product": np.random.choice(["A", "B", "C"], n),
    "revenue": np.random.uniform(10, 500, n),
    "quantity": np.random.randint(1, 20, n)
})
df_sales.head()

store_id,product,revenue,quantity
i32,str,f64,i32
7,"""C""",70.379101,15
4,"""C""",431.113739,5
8,"""C""",401.892737,13
5,"""A""",100.446713,8
7,"""B""",154.686362,8


## 1. Agrupaciones y Agregaciones
Polars permite realizar múltiples agregaciones de forma extremadamente eficiente.

In [2]:
df_agg = df_sales.group_by("store_id").agg([
    pl.col("revenue").sum().alias("total_revenue"),
    pl.col("quantity").mean().alias("avg_quantity"),
    pl.col("product").n_unique().alias("unique_products"),
    # Agregación condicional
    pl.col("revenue").filter(pl.col("product") == "A").sum().alias("revenue_A")
]).sort("total_revenue", descending=True)

df_agg.head()

store_id,total_revenue,avg_quantity,unique_products,revenue_A
i32,f64,f64,u32,f64
1,34883.15896,10.154412,3,15241.160398
3,33705.899336,10.598361,3,10219.945932
5,28295.487902,9.853448,3,9975.470492
6,28138.236922,8.842593,3,7550.440777
8,27435.888619,10.035714,3,8629.861855


## 2. Joins y Uniones
El rendimiento de los joins en Polars es significativamente superior a Pandas gracias a su motor paralelo.

In [3]:
df_stores = pl.DataFrame({
    "store_id": np.arange(1, 11),
    "location": ["Madrid", "Barcelona", "Valencia", "Sevilla", "Bilbao", "Malaga", "Zaragoza", "Murcia", "Palma", "Las Palmas"]
})

df_complete = df_sales.join(df_stores, on="store_id", how="inner")
df_complete.head()

store_id,product,revenue,quantity,location
i32,str,f64,i32,str
7,"""C""",70.379101,15,"""Zaragoza"""
4,"""C""",431.113739,5,"""Sevilla"""
8,"""C""",401.892737,13,"""Murcia"""
5,"""A""",100.446713,8,"""Bilbao"""
7,"""B""",154.686362,8,"""Zaragoza"""


## 3. Reshaping: Pivot y Melt
Cambiar la forma de los datos es fundamental para reportes y dashboards.

In [4]:
# Pivot: De largo a ancho
df_pivot = df_sales.filter(pl.col("store_id") <= 3).pivot(
    values="revenue",
    index="store_id",
    on="product",
    aggregate_function="sum"
)
print("Pivot Table:")
print(df_pivot)

# Melt: De ancho a largo
df_melt = df_pivot.melt(id_vars="store_id", variable_name="product", value_name="revenue_total")
print("\nMelted Table:")
print(df_melt.head())

Pivot Table:
shape: (3, 4)
┌────────┬───────┬───────┬───────┐
│ store_ ┆ A     ┆ B     ┆ C     │
│ id     ┆ ---   ┆ ---   ┆ ---   │
│ ---    ┆ f64   ┆ f64   ┆ f64   │
│ i32    ┆       ┆       ┆       │
╞════════╪═══════╪═══════╪═══════╡
│ 3      ┆ 10219 ┆ 11654 ┆ 11831 │
│        ┆ .9459 ┆ .9093 ┆ .0441 │
│        ┆ 32    ┆ 02    ┆ 02    │
│ 2      ┆ 6214. ┆ 8641. ┆ 7735. │
│        ┆ 45714 ┆ 73439 ┆ 26514 │
│        ┆ 7     ┆ 3     ┆ 7     │
│ 1      ┆ 15241 ┆ 7485. ┆ 12156 │
│        ┆ .1603 ┆ 47622 ┆ .5223 │
│        ┆ 98    ┆       ┆ 42    │
└────────┴───────┴───────┴───────┘

Melted Table:
shape: (5, 3)
┌──────────┬─────────┬───────────┐
│ store_id ┆ product ┆ revenue_t │
│ ---      ┆ ---     ┆ otal      │
│ i32      ┆ str     ┆ ---       │
│          ┆         ┆ f64       │
╞══════════╪═════════╪═══════════╡
│ 3        ┆ A       ┆ 10219.945 │
│          ┆         ┆ 932       │
│ 2        ┆ A       ┆ 6214.4571 │
│          ┆         ┆ 47        │
│ 1        ┆ A       ┆ 15241.160 │

C:\Users\miguel.vazquez\AppData\Local\Temp\ipykernel_35160\84800052.py:12: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_melt = df_pivot.melt(id_vars="store_id", variable_name="product", value_name="revenue_total")


## Resumen
- El `group_by` es paralelo por defecto.
- Los Joins son extremadamente rápidos.
- Pivot y Melt tienen una sintaxis más limpia y predecible.